In [10]:
##############################

# 1. Single Tracking 

##############################

import cv2
import numpy as np
import os
import sys


i = int(input("Please Choose a Number Between 0 and 6 !\nNumber : \n"))

if i > 6:
    print("Not Existing Tracking Method !!!")
    i = 0
    print("Default Tracker 'BOOSTING' ")
    print()
    
tracker_types = ["BOOSTING","MIL","KCF","TLD","MEDIANFLOW","MOSSE","CSRT"]
tracker_type = tracker_types[i]

print("Tracker Type : ", tracker_type)

if tracker_type == "BOOSTING":
    tracker = cv2.legacy.TrackerBoosting_create()
    
elif tracker_type == "MIL":
    tracker = cv2.legacy.TrackerMIL_create()

elif tracker_type == "KCF":
    tracker = cv2.legacy.TrackerKCF_create()

elif tracker_type == "TLD":
    tracker = cv2.legacy.TrackerTLD_create()

elif tracker_type == "MEDIANFLOW":
    tracker = cv2.legacy.TrackerMedianFlow_create()

elif tracker_type == "MOSSE":
    tracker = cv2.legacy.TrackerMOSSE_create()

elif tracker_type == "CSRT":
    tracker = cv2.legacy.TrackerCSRT_create()
    
    
#print(tracker)

cap = cv2.VideoCapture("Videos/race.mp4")

if not cap.isOpened():
    print("Error While Loading The Video !")
    sys.exit()
    
conn, frame = cap.read()

if not conn:
    print("Error While Loading The Frame !")
    sys.exit()
    
bbox = cv2.selectROI(frame)
print("Box : ",bbox )

tracker.init(frame,bbox)

colors = (np.random.randint(0,255),np.random.randint(0,255),np.random.randint(0,255))
print("Colors :", colors)

while True:
    
    conn, frame = cap.read()
    
    if not conn:
        break
        
    ok , bbox = tracker.update(frame)
    
    if ok:
        
        (x,y,w,h) = [int(v) for v in bbox]
        
        cv2.rectangle(frame, (x,y),(x+w,y+h),colors,2)
        
    else:
        
        cv2.putText(frame, "Tracking Failure!", (500,40),cv2.FONT_HERSHEY_COMPLEX, .75, (0,0,255))
    
    cv2.putText(frame, f"Tracker : {tracker_type}", (500,80),cv2.FONT_HERSHEY_COMPLEX, .75, (0,0,255))
    
    cv2.imshow("Single Tracker", frame)
    
    k = cv2.waitKey(1) & 0xFF 
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

Please Choose a Number Between 0 and 6 !
Number : 
5
Tracker Type :  MOSSE
Box :  (662, 270, 74, 137)
Colors : (198, 252, 25)


## 

In [4]:
##############################

# 2. Multiple Tracking 

##############################

import cv2
import numpy as np
from numpy.random import randint

tracker_types = ["BOOSTING","MIL","KCF","TLD","MEDIANFLOW","MOSSE","CSRT"]

def create_tracker_by_name(tracker_type):
    
    if tracker_type == tracker_types[0]:
        tracker = cv2.legacy.TrackerBoosting_create()

    elif tracker_type == tracker_types[1]:
        tracker = cv2.legacy.TrackerMIL_create()

    elif tracker_type == tracker_types[2]:
        tracker = cv2.legacy.TrackerKCF_create()

    elif tracker_type == tracker_types[3]:
        tracker = cv2.legacy.TrackerTLD_create()

    elif tracker_type == tracker_types[4]:
        tracker = cv2.legacy.TrackerMedianFlow_create()

    elif tracker_type == tracker_types[5]:
        tracker = cv2.legacy.TrackerMOSSE_create()

    elif tracker_type == tracker_types[6]:
        tracker = cv2.legacy.TrackerCSRT_create()
        
    else:
        
        tracker = None
        print("Invalid Name ! Avaible Trackers : \n")
        for t in tracker_types:
            print(t)
        print()
        print("---")
        
    return tracker

#print(create_tracker_by_name("KCF"))
#print(create_tracker_by_name("KCF123"))

cap = cv2.VideoCapture("Videos/race.mp4")

if not cap.isOpened():
    
    print("Error While Loading The Video !")
    sys.exit()
    
conn, frame = cap.read()

if not conn:
    
    print("Error While Loading The Frame !")
    sys.exit()
    
bboxes = []
colors = []

while True:
    
    bbox = cv2.selectROI("MultiTracker",frame)
    bboxes.append(bbox)
    colors.append(( randint(0, 255), randint(0,255), randint(0 ,255)))
    
    print("Press Q to Quit And Start Tracking")
    print("Press Any Other Key To Select The Next Object")
    
    k = cv2.waitKey(0) & 0xFF
    if k == ord("q"):
        break
        
print()
print("Boxes : ", bboxes)
print("Colors : ", colors)

tracker_type = "CSRT"
multi_tracker = cv2.legacy.MultiTracker_create()

for bbox in bboxes:
    
    multi_tracker.add(create_tracker_by_name(tracker_type), frame , bbox)
    
while cap.isOpened():
    
    conn, frame = cap.read()
    
    if not conn :
        break
        
    ok, boxes = multi_tracker.update(frame)
    
    if ok:
        
        for i,new_box in enumerate(boxes):
            
            (x,y,w,h) = [int(v) for v in new_box]
            cv2.rectangle(frame, (x,y),(x+w, y+h),colors[i],2)
            
    else:
        
        cv2.putText(frame, "Tracking Failure",(500,40),cv2.FONT_HERSHEY_COMPLEX,.75,(0,0,255))
        
    
    cv2.putText(frame, f"Tracker : {tracker_type}",(500,80),cv2.FONT_HERSHEY_COMPLEX,.75,(0,0,255))
    
    cv2.imshow("Multiple Tracking", frame)
    
    k = cv2.waitKey(1) & 0xFF 
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()

Press Q to Quit And Start Tracking
Press Any Other Key To Select The Next Object
Press Q to Quit And Start Tracking
Press Any Other Key To Select The Next Object
Press Q to Quit And Start Tracking
Press Any Other Key To Select The Next Object
Press Q to Quit And Start Tracking
Press Any Other Key To Select The Next Object

Boxes :  [(404, 277, 114, 164), (654, 237, 98, 216), (839, 211, 104, 213), (962, 217, 98, 212)]
Colors :  [(219, 6, 209), (227, 130, 213), (8, 93, 191), (188, 142, 215)]


## 

In [3]:
##############################

# 3. Goturn Tracking 

##############################

import cv2
import sys
from numpy.random import randint
import os
import numpy as np

if not (os.path.isfile("goturn.caffemodel") and os.path.isfile("goturn.prototxt")):
    print("Error Loading The Network Files !")
    sys.exit()
    
tracker = cv2.TrackerGOTURN_create()
tracker_type = type(tracker).__name__

cap = cv2.VideoCapture("Videos/race.mp4")

if not cap.isOpened():
    print("Error While Loading The Video !")
    sys.exit()
    
conn, frame = cap.read()

if not conn:
    print("Error While Loading The Frame !")
    sys.exit()
    
bbox = cv2.selectROI(frame)

tracker.init(frame,bbox)

colors = ( randint(0,255), randint(0,255), randint(0,255) )

while True:
    
    conn, frame = cap.read()
    
    if not conn:
        break
        
    ok , bbox = tracker.update(frame)
    
    if ok:
        
        (x,y,w,h) = [int(v) for v in bbox]
        cv2.rectangle(frame, (x,y), (x+w, y+h), colors, 2)
        
    else:

        
        cv2.putText(frame, "Tracking Failure!", (500,40), cv2.FONT_HERSHEY_COMPLEX, .75, (0,0,255))
        
    
    cv2.putText(frame, f"Tracker : {tracker_type}",(500,80),cv2.FONT_HERSHEY_COMPLEX, .75, (0,0,255))
    

    cv2.imshow("Tracking", frame)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break
        
cap.release()
cv2.destroyAllWindows()